In [ ]:
from browser_use import Agent, ChatGoogle
from dotenv import load_dotenv
import os
from datetime import datetime
import asyncio
import re
import json

# Load GOOGLE_API_KEY from .env
load_dotenv()

# Initialize the Gemini model
llm = ChatGoogle(model="gemini-2.5-flash")

# Create the browsing agent with browser configuration
agent = Agent(
    task=(
        "Find upcoming events in the next two weeks that could impact the US stock market. "
        "Look for Federal Reserve meetings, earnings announcements from major companies, "
        "economic data releases, and other market-moving events. "
        "Summarize with date, event type, and why it matters for investors."
    ),
    llm=llm,
    browser_config={
        "headless": True,
        "browser_type": "chromium",
        "browser_timeout": 60,
        "viewport_size": {"width": 1280, "height": 720},
        "extra_chromium_args": [
            "--no-sandbox",
            "--disable-dev-shm-usage",
            "--disable-gpu",
            "--disable-extensions",
            "--disable-plugins",
            "--disable-images",
            "--disable-javascript",
        ]
    }
)

# ---- Helper functions for structuring ----
def extract_events_from_text(text: str):
    """
    Extract structured events (date, event, type, reason) from raw agent output text.
    Very simple regex/NLP pass – can be upgraded with LLM parsing.
    """
    events = []

    # Regex for dates like "Sept 18, 2025" or "September 10"
    date_pattern = r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*\.?\s+\d{1,2}(?:,\s*\d{4})?)'

    lines = text.splitlines()
    for line in lines:
        date_match = re.search(date_pattern, line, flags=re.IGNORECASE)
        if date_match:
            date = date_match.group(1).strip()
            # Try to infer event type
            if "Fed" in line or "FOMC" in line:
                etype = "Federal Reserve"
            elif "earnings" in line or "report" in line:
                etype = "Earnings"
            elif "CPI" in line or "inflation" in line:
                etype = "Economic Data"
            elif "jobs" in line or "employment" in line:
                etype = "Economic Data"
            else:
                etype = "Other"

            event = {
                "date": date,
                "event": line.strip(),
                "type": etype,
                "impact_reason": "Market-moving event (auto-extracted)"
            }
            events.append(event)

    return events


async def main():
    try:
        print("🚀 Starting market events analysis...")
        history = await asyncio.wait_for(agent.run(max_steps=25), timeout=300)

        md_path = "market_events_report.md"
        summary_path = "market_events_summary.txt"
        structured_path = "market_events_structured.json"

        # ---- Save final result into reports ----
        final_result = history.final_result() or ""

        with open(md_path, "w", encoding="utf-8") as f:
            f.write("# Market Events Agent Report\n\n")
            f.write(f"*Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n\n")
            f.write("## Final Result\n")
            f.write(final_result + "\n")

        with open(summary_path, "w", encoding="utf-8") as f:
            f.write("MARKET EVENTS SUMMARY\n")
            f.write("=" * 50 + "\n\n")
            f.write(final_result if final_result else "No results were generated by the agent.")

        print(f"✅ Reports saved: {md_path}, {summary_path}")

        # ---- Step 24–37: Structuring Events ----
        structured_events = extract_events_from_text(final_result)

        with open(structured_path, "w", encoding="utf-8") as f:
            json.dump(structured_events, f, indent=2)

        print(f"📦 Structured JSON saved to {structured_path}")
        print(json.dumps(structured_events, indent=2))

    except Exception as e:
        print(f"❌ Error occurred: {str(e)}")

# For Jupyter / Async environments
await main()
